# Reading, cleaningm, scaling, enconding and spliting

In [14]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import re 

def fullRead(pathToTable, sep, anthro = False):

  df_renamed = pd.read_csv(pathToTable, sep = sep, encoding = "latin_1")
  
  df_name = re.sub("_ord.csv","",(re.sub("data/", "" ,pathToTable)))
  # reading and merging    
  
  if anthro == True:
    df_anthro = pd.read_csv("data/chronicAnthropometricCardiovascularData.csv", sep=";", decimal=",")
    df_renamed = df_renamed.merge(df_anthro)

    # separating by time moment and renaming

    df_renamed["Weight"] = ""
    df_renamed["BMI"] = ""
    df_renamed["Fat"] = ""
    df_renamed["CVRI"] = ""
    df_renamed["Bpmin"] = ""
    df_renamed["Bpmax"] = ""
    df_renamed["Frec"] = ""

    for i in range(len(df_renamed)):

        if df_renamed.loc[i]["Time"] == "Initial":
            df_renamed.loc[i,"Weight"] = df_renamed.loc[i]["Peso inicial"]
            df_renamed.loc[i,"BMI"] = df_renamed.loc[i]["IMC Inicial"]
            df_renamed.loc[i,"Fat"] = df_renamed.loc[i]["Grasa inicial"]
            df_renamed.loc[i,"CVRI"] = df_renamed.loc[i]["IRCV inicial"] 
            df_renamed.loc[i,"Bpmin"] = df_renamed.loc[i]["Bpmin inicial"] 
            df_renamed.loc[i,"Bpmax"] = df_renamed.loc[i]["Bpmax inicial"] 
            df_renamed.loc[i,"Frec"] = df_renamed.loc[i]["Frec inicial"] 
                
        if df_renamed.loc[i]["Time"] == "Final":
        
            df_renamed.loc[i,"Weight"] = df_renamed.loc[i]["Peso final"]
            df_renamed.loc[i,"BMI"] = df_renamed.loc[i]["IMC Final"]
            df_renamed.loc[i,"Fat"] = df_renamed.loc[i]["Grasa final"]
            df_renamed.loc[i,"CVRI"] = df_renamed.loc[i]["IRCV Final"] 
            df_renamed.loc[i,"Bpmin"] = df_renamed.loc[i]["Bpmin final"] 
            df_renamed.loc[i,"Bpmax"] = df_renamed.loc[i]["Bpmax final"] 
            df_renamed.loc[i,"Frec"] = df_renamed.loc[i]["Frec final"] 
        
    df_renamed.drop(columns = ["Peso inicial", "Peso final", "Delta Peso", "Talla", "IMC Inicial", "IMC Final", "Delta IMC", "Grasa inicial", "Grasa final", "Delta Grasa", "IRCV Final", "IRCV inicial", "Bpmin final", "Bpmin inicial", "Bpmax final", "Bpmax inicial", "Frec final", "Frec inicial",], inplace=True )
  
  df_renamed.drop(columns = ["Unnamed: 0", "grouping"], inplace=True )
  df_renamed.fillna(0, inplace=True)
  return (df_renamed, df_name)

def scaling(df_read):
   
   scaler = preprocessing.MinMaxScaler()
   numCols = df_read.select_dtypes(include=np.number).drop("numVol",1).columns
   df_read[numCols] = scaler.fit_transform(df_read[numCols])
   return df_read

def encodingSplitting(df):
  df = pd.get_dummies(df, columns = ["Sweetener", "Sex", "Time"], drop_first=False)
  X_met, y_met = df[df["Time_Initial"] == 1].drop(["numVol", "Time_Initial", "Time_Final"], axis=1), df[df["Time_Final"] == 1].drop(['Sweetener_SA', 'Sweetener_ST','Sweetener_SU', 'Sex_MAN', 'Sex_WOMAN', 'Time_Final', 'Time_Initial','numVol', 'Weight','BMI', 'Fat', 'CVRI', 'Bpmin', 'Bpmax', 'Frec'], axis = 1)
  X_metTrain, X_metTest, y_metTrain, y_metTest = train_test_split(X, y, test_size=0.2, random_state=42)

  X_full, y_full = df[df["Time_Initial"] == 1].drop(["numVol", "Time_Initial", "Time_Final"], axis=1), df[df["Time_Final"] == 1].drop(['numVol','Sweetener_SA', 'Sweetener_ST','Sweetener_SU','Time_Final', 'Time_Initial'], axis = 1)
  X_fullTrain, X_fullTest, y_fullTrain, y_fullTest = train_test_split(X_full, y_full, test_size=0.3, random_state=42)

  return(X_met, y_met, X_metTrain, X_metTest, y_metTrain, y_metTest, X_full, y_full, X_fullTrain, X_fullTest, y_fullTrain, y_fullTest)


#df_PF = pd.get_dummies(scaling(fullRead("data/plasmFlav_ord.csv",  sep = ",", anthro= True)), columns = ["Sweetener", "Sex", "Time"], drop_first=False)
#df_PA = scaling(fullRead("data/plasmAnt_ord.csv",  sep = ",", anthro= True))
#df_UF = scaling(fullRead("data/urineFlav_ord.csv",  sep = ",", anthro= True))
#df_UA = scaling(fullRead("data/urineAnt_ord.csv",  sep = ",", anthro= True))
#X_test.to_csv("X_met_test_urineAnt.csv", index=False)
#X_fulltest.to_csv("X_full_test_urineAnt.csv",index=False)

In [39]:
X_met

NameError: name 'X_met' is not defined

# Running models

Random Forest function

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
# import pickle

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# define model

def randomForestReg(df, df_name, met = True):
    
    if (met):
        
        X_met, y_met, X_metTrain, X_metTest, y_metTrain, y_metTest, X_full, y_full, X_fullTrain, X_fullTest, y_fullTrain, y_fullTest = encodingSplitting(df)

        model = RandomForestRegressor()

        grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

        grid_search.fit(X_metTrain, y_metTrain)
        best_grid = grid_search.best_estimator_
        # define the evaluation procedure
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        # evaluate the model and collect the scores
        n_scores = cross_val_score(best_grid, X_metTest, y_metTest, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
        # force the scores to be positive
        n_scores = np.absolute(n_scores)
        # summarize performance

        #filename = 'rf_met_plasmAnt.pkl'
        #with open(filename, 'wb') as file:
        #    pickle.dump(grid_search.best_estimator_, file)

        print('Only Metabolic model ' + df_name + ': MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
        MAE = (np.mean(n_scores), np.std(n_scores))
        return(best_grid, MAE)

    else:
            
        X_met, y_met, X_metTrain, X_metTest, y_metTrain, y_metTest, X_full, y_full, X_fullTrain, X_fullTest, y_fullTrain, y_fullTest = encodingSplitting(df)

        model = RandomForestRegressor()

        grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                        cv = 3, n_jobs = -1, verbose = 2)

        grid_search.fit(X_fullTrain, y_fullTrain)
        best_grid = grid_search.best_estimator_
        # define the evaluation procedure
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        # evaluate the model and collect the scores
        n_scores = cross_val_score(best_grid, X_fullTest, y_fullTest, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
        # force the scores to be positive
        n_scores = np.absolute(n_scores)
        # summarize performance

        #filename = 'rf_met_plasmAnt.pkl'
        #with open(filename, 'wb') as file:
        #    pickle.dump(grid_search.best_estimator_, file)

        print('Full model ' + df_name + ': MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
        MAE = (np.mean(n_scores), np.std(n_scores))
        return(best_grid, MAE)



Random Forest Function

In [33]:
paths = ["plasmFlav_ord.csv", "plasmAnt_ord.csv", "urineFlav_ord.csv", "urineAnt_ord.csv"]
paths = ["data/" + s for s in paths]

resultsModelsRF = {}
resultsScoresRF = {}

for path in paths:
    df, df_name = fullRead(path, sep = ",", anthro = True)
    resultsModelsRF["modelRFMet{0}".format(df_name)], resultsScoresRF["modelRFMet{0}".format(df_name)] = randomForestReg(scaling(df), df_name, met = True)
    resultsModelsRF["modelRFFull{0}".format(df_name)], resultsScoresRF["modelRFFull{0}".format(df_name)] = randomForestReg(scaling(df), df_name, met = False)



#modelRFMet = randomForestReg(scaling(fullRead("data/plasmFlav_ord.csv",  sep = ",", anthro= True)[0]), met = True)
#modelRFFull = randomForestReg(scaling(fullRead("data/plasmFlav_ord.csv",  sep = ",", anthro= True)), met = False)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
Only Metabolic plasmFlav: MAE: 0.053 (0.019)
Fitting 3 folds for each of 288 candidates, totalling 864 fits
Full model plasmFlav: MAE: 0.094 (0.010)
Fitting 3 folds for each of 288 candidates, totalling 864 fits
Only Metabolic plasmAnt: MAE: 0.053 (0.020)
Fitting 3 folds for each of 288 candidates, totalling 864 fits
Full model plasmAnt: MAE: 0.123 (0.010)
Fitting 3 folds for each of 288 candidates, totalling 864 fits
Only Metabolic urineFlav: MAE: 0.054 (0.020)
Fitting 3 folds for each of 288 candidates, totalling 864 fits
Full model urineFlav: MAE: 0.091 (0.013)
Fitting 3 folds for each of 288 candidates, totalling 864 fits
Only Metabolic urineAnt: MAE: 0.054 (0.020)
Fitting 3 folds for each of 288 candidates, totalling 864 fits
Full model urineAnt: MAE: 0.093 (0.008)


In [35]:
resultsScores

{'modelRFMetdata/plasmFlav_ord.csv': (0.05305571020940363,
  0.019355957726905838),
 'modelRFFulldata/plasmFlav_ord.csv': (0.09350444273592906,
  0.00974506016507601),
 'modelRFMetdata/plasmAnt_ord.csv': (0.05349754173439036, 0.02006719966152714),
 'modelRFFulldata/plasmAnt_ord.csv': (0.12341770738231064,
  0.010457016301911155),
 'modelRFMetdata/urineFlav_ord.csv': (0.0537300603471775,
  0.019774406267297245),
 'modelRFFulldata/urineFlav_ord.csv': (0.09082384827131884,
  0.013140742577472485),
 'modelRFMetdata/urineAnt_ord.csv': (0.05414192453820145,
  0.019888635239077763),
 'modelRFFulldata/urineAnt_ord.csv': (0.09300592377394583,
  0.007926720376069184)}

MultiLayerPerceptron Function

In [49]:
# mlp for multi-output regression

import numpy as np
import tensorflow as tf
# import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score, RepeatedKFold, GridSearchCV

tf.get_logger().setLevel('ERROR')
# fix random seed for reproducibility

def MLPReg (df, df_name, met):
    
    if (met):
        seed = 7
        tf.random.set_seed(seed)

        X_met, y_met, X_metTrain, X_metTest, y_metTrain, y_metTest, X_full, y_full, X_fullTrain, X_fullTest, y_fullTrain, y_fullTest = encodingSplitting(df)


        epochs = [10, 50, 100]
        batch_size = [10, 20, 40, 60, 80, 100]

        param_grid = dict(batch_size=batch_size, epochs=epochs)

        # get the model
        def get_model(n_inputs, n_outputs):
            model_nn = Sequential()
            model_nn.add(Dense(64, input_shape=(n_inputs,),activation="relu"))
            model_nn.add(Dropout(0.5))
            model_nn.add(Dense(32, activation="relu"))
            model_nn.add(Dropout(0.5))
            model_nn.add(Dense(n_outputs, activation='linear'))
            model_nn.compile(loss='mae', optimizer=tf.keras.optimizers.Adam())

            return model_nn
        
        # evaluate a model using repeated k-fold cross-validation
        def evaluate_model(X, y):
            results = list()
            n_inputs, n_outputs = X.shape[1], y.shape[1]
            # define evaluation procedure
            cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            # define modeld
            model_nn = KerasRegressor(model = get_model(n_inputs, n_outputs), optimizer=tf.keras.optimizers.Adam(), verbose=0)
            # fit model
            grid = GridSearchCV(estimator=model_nn, param_grid=param_grid, n_jobs=-1, cv=3, verbose=0)
            grid_result = grid.fit(X_train, y_train) # evaluate model on test set
            # summarize results
            best_grid = grid_result.best_estimator_
            # define the evaluation procedure
            cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
            # evaluate the model and collect the scores
            n_scores = cross_val_score(best_grid, X_test, y_test, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
            # force the scores to be positive
            n_scores = np.absolute(n_scores)

            # store the model
            # filename = 'mlp_met_plasmAnt.pkl'
            # with open(filename, 'wb') as file:
            #    pickle.dump(grid_result.best_estimator_, file)

            # summarize performance
            print('Only Metabolic model ' + df_name +'MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
            MAE = (np.mean(n_scores), np.std(n_scores))
            return(best_grid, MAE)
            # evaluate model
            
        modelMLP, MAE = evaluate_model(X_met, y_met)
    
    else:
        seed = 7
        tf.random.set_seed(seed)

        X_met, y_met, X_metTrain, X_metTest, y_metTrain, y_metTest, X_full, y_full, X_fullTrain, X_fullTest, y_fullTrain, y_fullTest = encodingSplitting(df)


        epochs = [10, 50, 100]
        batch_size = [10, 20, 40, 60, 80, 100]

        param_grid = dict(batch_size=batch_size, epochs=epochs)

        # get the model
        def get_model(n_inputs, n_outputs):
            model_nn = Sequential()
            model_nn.add(Dense(64, input_shape=(n_inputs,),activation="relu"))
            model_nn.add(Dropout(0.5))
            model_nn.add(Dense(32, activation="relu"))
            model_nn.add(Dropout(0.5))
            model_nn.add(Dense(n_outputs, activation='linear'))
            model_nn.compile(loss='mae', optimizer=tf.keras.optimizers.Adam())

            return model_nn
        
        # evaluate a model using repeated k-fold cross-validation
        def evaluate_model(X, y):
            results = list()
            n_inputs, n_outputs = X.shape[1], y.shape[1]
            # define evaluation procedure
            cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            # define modeld
            model_nn = KerasRegressor(model = get_model(n_inputs, n_outputs), optimizer=tf.keras.optimizers.Adam(), verbose=0)
            # fit model
            grid = GridSearchCV(estimator=model_nn, param_grid=param_grid, n_jobs=-1, cv=3, verbose=0)
            grid_result = grid.fit(X_train, y_train) # evaluate model on test set
            # summarize results
            best_grid = grid_result.best_estimator_
            # define the evaluation procedure
            cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
            # evaluate the model and collect the scores
            n_scores = cross_val_score(best_grid, X_test, y_test, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
            # force the scores to be positive
            n_scores = np.absolute(n_scores)

            # store the model
            # filename = 'mlp_met_plasmAnt.pkl'
            # with open(filename, 'wb') as file:
            #    pickle.dump(grid_result.best_estimator_, file)

            # summarize performance
            print('Full model ' + df_name + 'MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
            MAE = (np.mean(n_scores), np.std(n_scores))
            return(best_grid, MAE)
            # evaluate model
            
        modelMLP, MAE = evaluate_model(X_full, y_full)    
    return (modelMLP, MAE)

MLP Execution

In [50]:
paths = ["plasmFlav_ord.csv", "plasmAnt_ord.csv", "urineFlav_ord.csv", "urineAnt_ord.csv"]
paths = ["data/" + s for s in paths]

resultsModelsMLP = {}
resultsScoresMLP = {}

for path in paths:
    df, df_name = fullRead(path, sep = ",", anthro = True)
    resultsModelsMLP["modelMLPMet{0}".format(df_name)], resultsScoresMLP["modelMLPMet{0}".format(df_name)] = MLPReg(scaling(df), df_name, met = True)
    resultsModelsMLP["modelMLPFull{0}".format(df_name)], resultsScoresMLP["modelMLPFull{0}".format(df_name)] = MLPReg(scaling(df), df_name, met = False)



MAE: 0.054 (0.022)
MAE: 0.092 (0.014)
MAE: 0.096 (0.019)
MAE: 0.111 (0.016)
MAE: 0.051 (0.021)
MAE: 0.080 (0.012)
MAE: 0.071 (0.019)
MAE: 0.087 (0.014)


In [52]:
resultsScoresMLP

{'modelMLPMetdata/plasmFlav_ord.csv': (0.053883654018792346,
  0.02160481184928749),
 'modelMLPFulldata/plasmFlav_ord.csv': (0.09196652860139162,
  0.01363769383341583),
 'modelMLPMetdata/plasmAnt_ord.csv': (0.09566204001361353,
  0.01885550970993914),
 'modelMLPFulldata/plasmAnt_ord.csv': (0.11069668151990246,
  0.015924388417305948),
 'modelMLPMetdata/urineFlav_ord.csv': (0.0513886545338527,
  0.020606704620211604),
 'modelMLPFulldata/urineFlav_ord.csv': (0.07958265553535816,
  0.011993007172429588),
 'modelMLPMetdata/urineAnt_ord.csv': (0.07092841537529868,
  0.01949563980533962),
 'modelMLPFulldata/urineAnt_ord.csv': (0.0870458449412254,
  0.014102588320115112)}

XGBoost function

In [71]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score, GridSearchCV

def XGBReg (df, df_name, met):

    X_met, y_met, X_metTrain, X_metTest, y_metTrain, y_metTest, X_full, y_full, X_fullTrain, X_fullTest, y_fullTrain, y_fullTest = encodingSplitting(df)

    if (met):
        
        xgbReg = XGBRegressor()

        param_grid = {'max_depth'        : [None, 1, 3, 5, 10, 20],
                    'subsample'        : [0.5, 1],
                    'learning_rate'    : [0.001, 0.01, 0.1],
                    'booster'          : ['gbtree', 'gblinear', 'dart']
                    }


        grid_search = GridSearchCV(estimator = xgbReg, param_grid = param_grid, cv= 3, n_jobs=-1,
                                verbose=2)

        grid_search.fit(X_metTrain, y_metTrain)
        best_grid = grid_search.best_estimator_


        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        n_scores = cross_val_score(best_grid, X_metTest, y_metTest,  scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
        n_scores = np.absolute(n_scores)

        print("Only metabolic model " + df_name +' MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
        MAE = (np.mean(n_scores), np.std(n_scores))
        return(best_grid, MAE)
    
    else:
        xgbReg = XGBRegressor()

        param_grid = {'max_depth'        : [None, 1, 3, 5, 10, 20],
                    'subsample'        : [0.5, 1],
                    'learning_rate'    : [0.001, 0.01, 0.1],
                    'booster'          : ['gbtree', 'gblinear', 'dart']
                    }


        grid_search = GridSearchCV(estimator = xgbReg, param_grid = param_grid, cv= 3, n_jobs=-1,
                                verbose=2)

        grid_search.fit(X_fullTrain, y_fullTrain)
        best_grid = grid_search.best_estimator_

        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        n_scores = cross_val_score(best_grid, X_fullTest, y_fullTest,  scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
        n_scores = np.absolute(n_scores)

        print("Full model "+ df_name + ' MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
        MAE = (np.mean(n_scores), np.std(n_scores))
        return(best_grid, MAE)        


XGBoost Execution

In [72]:
paths = ["plasmFlav_ord.csv", "plasmAnt_ord.csv", "urineFlav_ord.csv", "urineAnt_ord.csv"]
paths = ["data/" + s for s in paths]

resultsModelsXGB = {}
resultsScoresXGB = {}

for path in paths:
    df, df_name = fullRead(path, sep = ",", anthro = True)
    resultsModelsXGB["modelXGBMet{0}".format(df_name)], resultsScoresXGB["modelXGBMet{0}".format(df_name)] = XGBReg(scaling(df), df_name, met = True)
    resultsModelsXGB["modelXGBFull{0}".format(df_name)], resultsScoresXGB["modelXGBFull{0}".format(df_name)] = XGBReg(scaling(df), df_name, met = False)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Only metabolic model plasmFlav MAE: 0.066 (0.020)
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Full model plasmFlav MAE: 0.104 (0.009)
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Only metabolic model plasmAnt MAE: 0.066 (0.020)
Fitting 3 folds for each of 108 candidates, totalling 324 fits


KeyboardInterrupt: 

In [59]:
resultsScoresXGB

{'modelXGBMetplasmFlav': (0.06566482327975234, 0.020082282056815344),
 'modelXGBFullplasmFlav': (0.05899397690121679, 0.01190496238480627),
 'modelXGBMetplasmAnt': (0.06566482327975234, 0.020082282056815344),
 'modelXGBFullplasmAnt': (0.12116975738171971, 0.01455740739431039),
 'modelXGBMeturineFlav': (0.06566482327975234, 0.020082282056815344),
 'modelXGBFullurineFlav': (0.08880958387223581, 0.02758939798969107),
 'modelXGBMeturineAnt': (0.06566482327975234, 0.020082282056815344),
 'modelXGBFullurineAnt': (0.0778253075732373, 0.010167032490283107)}

# Feature Selection experiments

SelectFromModel method

In [76]:
resultsModels["modelRFMetdata/plasmFlav_ord.csv"]

RandomForestRegressor(max_depth=110, max_features=2, min_samples_leaf=3,
                      min_samples_split=12, n_estimators=300)

In [82]:
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.ensemble import RandomForestRegressor


df, df_name = fullRead("data/plasmFlav_ord.csv", sep = ",", anthro = True)
X_met, y_met, X_metTrain, X_metTest, y_metTrain, y_metTest, X_full, y_full, X_fullTrain, X_fullTest, y_fullTrain, y_fullTest = encodingSplitting(df)

model = resultsModels["modelRFMetdata/plasmFlav_ord.csv"] # model PF_met

#model.fit(X_metTrain, y_metTrain)

thresholds = np.sort(model.feature_importances_)
results = pd.DataFrame()
for thresh in thresholds:
 # select features using threshold
 selection = SelectFromModel(model, threshold=thresh, prefit=True)
 feature_idx = selection.get_support()
 print(X_metTrain.columns[feature_idx])
 select_X_train = selection.transform(X_metTrain)
 # train model
 selection_model = model
 selection_model.fit(select_X_train, y_metTrain)
 print (selection_model)
 # eval model
 select_X_test = selection.transform(X_metTest)
 #thresholds
 y_pred = selection_model.predict(select_X_test)
 predictions = [value for value in y_pred]
 mae = mean_absolute_error(y_metTest, predictions)
 result1 = pd.DataFrame({"thresh": thresh, "n":select_X_train.shape[1], "MAE":mae}, index = [30 - select_X_train.shape[1]])
 results = pd.concat([results, result1])
 #print("Thresh=%.3f, n=%d, MAE: %.4f" % (thresh, select_X_train.shape[1], accuracy))

print(results)

thresholdSelected = results.sort_values(by= "MAE", ascending = True).head(n=1)["thresh"]

### Si hay que reentrenar:
# select features using threshold
selection = SelectFromModel(model, threshold=thresholdSelected, prefit=True)
feature_idx = selection.get_support()
print(X_metTrain.columns[feature_idx])
select_X_train = selection.transform(X_metTrain)
# train model
model.fit(select_X_train, y_metTrain)
# eval model
select_X_test = selection.transform(X_metTest)
y_pred = selection_model.predict(select_X_test)
predictions = [value for value in y_pred]
mae = mean_absolute_error(y_metTest, predictions)
result1 = pd.DataFrame({"thresh": thresh, "n":select_X_train.shape[1], "MAE":mae}, index = [30 - select_X_train.shape[1]])
result1

### Si ->NO<- hay que reentrenar


Index(['E', 'E.S', 'Total.E', 'HE.G', 'N.G', 'Weight', 'BMI', 'Fat', 'CVRI',
       'Bpmin', 'Bpmax', 'Frec', 'Sweetener_SA', 'Sweetener_ST',
       'Sweetener_SU', 'Sex_MAN', 'Sex_WOMAN'],
      dtype='object')
RandomForestRegressor(max_depth=110, max_features=2, min_samples_leaf=3,
                      min_samples_split=12, n_estimators=300)
Index(['E', 'E.S', 'Total.E', 'HE.G', 'N.G', 'Weight', 'BMI', 'Fat', 'CVRI',
       'Bpmin', 'Bpmax', 'Frec', 'Sweetener_SA', 'Sweetener_ST',
       'Sweetener_SU', 'Sex_MAN', 'Sex_WOMAN'],
      dtype='object')
RandomForestRegressor(max_depth=110, max_features=2, min_samples_leaf=3,
                      min_samples_split=12, n_estimators=300)


ValueError: X has 16 features, but RandomForestRegressor is expecting 17 features as input.

In [78]:
results

,thresh,n,MAE
13,0.007003,17,0.057547
14,0.010890,16,0.059614
15,0.013178,15,0.058458
16,0.017247,14,0.060473
17,0.017888,13,0.056218
18,0.022161,12,0.059634
19,0.039065,11,0.058788
20,0.056996,10,0.058360
21,0.061445,9,0.059553
22,0.066593,8,0.058089


In [79]:
thresholdSelected = results.sort_values(by= "MAE", ascending = True).head(n=1)["thresh"]

# Si hay que reentrenar:



# select features using threshold
selection = SelectFromModel(model, threshold=thresholdSelected, prefit=True)
feature_idx = selection.get_support()
print(X_train.columns[feature_idx])
select_X_train = selection.transform(X_train)
# train model
selection_model = RandomForestRegressor()
selection_model.fit(select_X_train, y_train)


Index(['HE.G', 'N.G', 'Weight', 'BMI', 'Fat', 'Bpmin', 'Frec'], dtype='object')


RandomForestRegressor()

In [ ]:
import eli5 
from eli5.sklearn import PermutationImportance
perm_base = PermutationImportance(modelMLPMet, random_state = 1).fit(X, y)
eli5.show_weights(perm_base, feature_names = X.columns.tolist())

ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (c:\Users\dres2\anaconda3\Lib\site-packages\sklearn\utils\metaestimators.py)

In [22]:
import eli5 
from eli5.sklearn import PermutationImportance
perm_base = PermutationImportance(uwu.best_estimator_, random_state = 1).fit(X_fulltrain, y_fulltrain)
eli5.show_weights(perm_base, feature_names = X_fulltrain.columns.tolist())

Weight,Feature
0.0905 ± 0.0166,Sex
0.0336 ± 0.0154,Sweetener
0.0169 ± 0.0053,Weight
0.0153 ± 0.0060,BMI
0.0080 ± 0.0031,Fat
0.0050 ± 0.0025,CVRI
0.0026 ± 0.0016,Bpmin
0.0026 ± 0.0009,Bpmax
0.0024 ± 0.0019,VA.GS
0.0022 ± 0.0016,CA


In [ ]:
# Full parameters gridsearch
# DONT EXECUTE
'''
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV

from scikeras.wrappers import KerasRegressor
import tensorflow as tf


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]

param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer, optimizer__learning_rate=learn_rate, optimizer__momentum=momentum,
                  model__init_mode=init_mode, model__activation=activation, model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

# get the model
def get_model(n_inputs, n_outputs, dropout_rate, weight_constraint, activation="relu", init_mode='uniform'):
    model_nn = Sequential()
    model_nn.add(Dense(64, input_shape=(X.shape[1],),activation=activation, kernel_initializer=init_mode, kernel_constraint=MaxNorm(weight_constraint)))
    model_nn.add(Dropout(dropout_rate))
    model_nn.add(Dense(32, activation=activation, kernel_initializer=init_mode))
    model_nn.add(Dropout(dropout_rate))
    model_nn.add(Dense(y.shape[1], activation='linear', kernel_initializer=init_mode))
    return model_nn
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
 results = list()
 n_inputs, n_outputs = X.shape[1], y.shape[1]
 # define evaluation procedure
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# define modeld
 model_nn = KerasRegressor(model = get_model) #(n_inputs, n_outputs, activation="relu", init_mode='uniform'))
 # fit model
 grid = GridSearchCV(estimator=model_nn, param_grid=param_grid, n_jobs=-1, cv=3)
 grid_result = grid.fit(X_train, y_train) # evaluate model on test set
 mae = model_nn.evaluate(X_test, y_test)
 # store result
 print('>%.3f' % mae)
 results.append(mae)
 return results
 
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))
'''